# Programa que calcula previsão para o dia seguinte

In [1]:
print('ola mundo')
import tensorflow as tf
print(tf.__version__)

ola mundo


2022-11-08 23:56:18.771984: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-08 23:56:18.780218: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.7.0


In [2]:
import requests
import pandas as pd
import numpy as np

In [3]:
urlRedes = "http://vps-40d69db1.vps.ovh.ca:23004/api/TreinoRedes/obtemListaParaPrevisao"
urlExemplos = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoEntradas/obtemProximoDia"
urlPrevisao = 'http://vps-40d69db1.vps.ovh.ca:23004/api/PrevisaoRedes/recebePrevisaoTreino'

In [4]:
def incluiEntrada(treino):
    urlEntrada = "http://vps-40d69db1.vps.ovh.ca:23004/api/CotacaoIntradayAcaoResultados/listaPrecoEntradaPrevisaoProximoB3?idRegra=" + str(treino['regraProjecaoId']) + "&idGrupo=" + str(treino['grupoAcaoId'])
    print(urlEntrada)
    r = requests.get(urlEntrada)
    j = r.json()
    dfEntradas = pd.DataFrame.from_dict(j)
    print(dfEntradas.head())
    return dfEntradas

In [5]:
def trataRede(rede):
    print('ID:' , rede['id'])
    nome_arquivo = '/root/executa-rede/redes/treino_rede' + str(rede['id']) + '.h5'
    print(nome_arquivo)
    modelo = tf.keras.models.load_model(nome_arquivo)
    modelo.summary()
    result = modelo.predict(X)
    dfDados['resultado'] = result
    dfDados['treinoRedeId'] = str(rede['id'])
    dfDados['tipoCompraVenda'] = rede['tipoCompraVenda']
    dfEntradas = incluiEntrada(rede)
    dfPrevisao = pd.merge(dfDados,dfEntradas, how='left', left_on='ticker' ,  right_on='ticker')
    json = dfPrevisao.to_dict(orient='list')
    response = requests.post(urlPrevisao, data=json)
    print(response)

In [6]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')

In [7]:
print('conexao redes')
r = requests.get(urlRedes)
j = r.json()
dfRede = pd.DataFrame.from_dict(j)
print(dfRede.head())

conexao redes
   id  grupoAcaoId  periodoTreinoRedeId tipoExemploTreinoId  regraProjecaoId  \
0  32           12                    1                None               57   
1  43           12                    1                None               68   
2  54           12                    1                None               43   
3  77           12                    1                None               66   
4  79           12                    1                None               68   

   redeNeuralId  ativoPrevisao  limiteParaEntrada  qtdeLucroTeste  \
0             2              1               0.47               9   
1             2              1               0.23               8   
2             1              1               0.30               8   
3             1              1               0.26               9   
4             1              1               0.23               9   

   qtdePrejuizoTeste  ...                 dataTeste  ativoPrevisaoTeste  \
0              

In [8]:
print('conexao exemplos')
print(urlExemplos)
r = requests.get(urlExemplos)
j = r.json()
dfDados = pd.DataFrame.from_dict(j)
print(dfDados.shape())

conexao exemplos
http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoEntradas/obtemProximoDia


TypeError: 'tuple' object is not callable

In [ ]:
dfTratadoX = dfDados.apply(trataLinhaX, axis='columns')
X = np.stack(dfTratadoX.values)
X.shape

In [ ]:
dfDados.drop('campoX', inplace=True, axis=1)

dfRede.apply(trataRede, axis='columns')